<div align="right"><i>Peter Norvig<br>2012<br>updated 2019, 2020</i></div>

# Weighing Twelve Balls on a Scale: 🎱🎱🎱🎱 ⚖ 🎱🎱🎱🎱 

> *You are given twelve identical-looking balls and a two-sided scale. One of the balls is of a different weight, although you don't know whether it is lighter or heavier. How can you use just three weighings of the scale to determine not only what the different ball is, but also whether it is lighter or heavier?*

This is [a classic](https://en.wikipedia.org/wiki/Balance_puzzle) brain-teaser puzzle (first appearing in 1945), meant to be solved with paper and pencil.  But I want to solve not just this specific puzzle, but related puzzles where you can vary (a) the number of balls, (b) the number of weighings allowed, and (c) whether the different ball might be heavier, lighter, or either, or neither. So I'll be better served by a computer program, not a pencil. 

(I originally wrote this program in Lisp around 1980, ported it to Python in 2012, and am republishing it here in revised form because the puzzle was mentioned in the [538 Riddler](https://fivethirtyeight.com/features/which-billiard-ball-is-rigged/) for 16 August 2019. It also appeared in a [Numberplay](https://wordplay.blogs.nytimes.com/2014/07/21/12coin/) column in 2014 (with coins instead of balls), in the Museum of Math's [Mindbenders](https://momath.org/civicrm/?page=CiviCRM&q=civicrm/event/info&reset=1&id=1620) series during shelter-in-place 2020, and in many other venues.)


# Defining the Vocabulary
    
Here are the concepts I'm dealing with:

- **balls**: In the general case we have `N` balls. I'll represent them with a list like `[1, 2, 3]` for `N = 3`.
- **oddball**: One of the balls is different in weight; I'll use the term **oddball** to indicate which ball is different, and how. I'll use `+N` to mean that ball `N` is heavier, and `-N` to mean ball `N` is lighter. (Note that `0` cannot be a ball.)
- **possible oddballs**: I'll keep track of the set of possible oddballs for a puzzle, and the remaining possibilities after each weighing. With `N = 3`, the set of all possible oddballs is `{+1, -1, +2, -2, +3, -3}`. 
- **puzzle**: I'll express the puzzle stated above with `Puzzle(N=12, weighings=3)`. A puzzle is defined by the balls involved, the number of weighings allowed, and the possible oddballs. (I'll add a few complications later.)
- **weighing**: I can weigh a collection of balls on the left versus a collection on the right, and the result will be that the left side is greater than, equal to, or less than the right in weight. I'll denote that with the call `weigh(L, R, oddball)`, which returns a string, `'gt'`, `'eq'`, or `'lt'`.
- **weight**: I'll arbitrarily say that a normal ball weighs 100, a lighter ball 99, and a heavier ball 101.
- **solution**: Given a puzzle, a solution is a **strategy tree** that can correctly discover the oddball, whatever it is, in the allowable number of weighings. Since a tree with W weighings has 3<sup>W</sup> possible outcomes (leaf nodes), we can deduce that it is impossible to find a solution in W weighings if there are more than 3<sup>W</sup> possible oddballs.
- **strategy tree**: A tree whose leaf nodes are all oddballs (ints), and whose interior nodes have 5 components:  `Node(L, R, gt, eq, lt)` is a node where `L` and `R` are the collections of balls to be weighed, and `gt`, `eq`, and `lt` are branches for subtrees corresponding to the three possible results of the weighing. (I'll also use `None` to mean "could not find a valid tree.")
- **following a path in a tree**: I'll use `follow(tree, oddball)` to say *follow the path through the tree, doing each weighing under the assumption of the given oddball, and return the leaf node reached by the path&mdash;the oddball that the tree predicts.* Note that the function `follow` gets to see what the oddball is, but the `tree` never gets direct access to that; the tree has to figure it out by doing weighings.
- **valid solution**: A tree is a valid solution to a puzzle if no branch uses more than the allowable number of weighings, and if, for every possible oddball in the puzzle, following the path through the tree correctly returns that oddball as the answer. I'll use `valid(tree, puzzle)` for this.

Note: In the definition of the `Puzzle` class, by default the possible oddballs are all the "lighter" and "heavier" versions of the puzzle's balls. But if you want a different kind of puzzle, the `Puzzle` class gives you two ways to say that: 
- You can specify the keyword argument `odd`, whose value can be any subset of `{-1, 0, +1}`, where `-1` means "any one ball might be lighter," `+1` means "any one ball might be heavier," and `0` means "it might be that no ball is odd; they might all weigh the same."
- You can specify the keyword argument `oddballs` and give an explicit set of oddballs; any set of positive, negative, or zero integers.


# Implementation of Data Types


In [1]:
from collections import namedtuple
import random

Ball = Oddball = Leaf = int

Node = namedtuple('Node', 'L, R, gt, eq, lt')

Tree = (Node, Leaf) # A tree can be an interior node or a leaf

class Puzzle:
    "Represent a specific ball-weighing puzzle."
    def __init__(self, N=12, weighings=3, odd={-1, +1}, oddballs=None):
        self.balls     = list(range(1, N + 1))
        self.weighings = weighings
        self.oddballs  = oddballs or {b * o for b in self.balls for o in odd}

# Implementation of Basic Functions

Here are all the straightforward utility functions; everything except the main function `find_tree`, that actually solves a puzzle.

In [2]:
def weigh(L, R, oddball) -> str:
    "Weigh balls L against balls R, given the oddball; return 'gt', 'eq', or 'lt'."
    diff = sum(weight(b, oddball) for b in L) - sum(weight(b, oddball) for b in R)
    return ('gt' if diff > 0 else
            'lt' if diff < 0 else
            'eq')

def weight(ball, oddball) -> int: 
    "How heavy is this ball (given that we know what the oddball is)?"
    return 101 if +ball == oddball else 99 if -ball == oddball else 100
    
def solve(puzzle) -> Tree or None:
    "Return a valid tree (one that solves the puzzle), or None for failure."
    tree = find_tree(puzzle, puzzle.oddballs, puzzle.weighings)
    return tree if valid(tree, puzzle) else None
    
def follow(tree, oddball) -> Oddball:
    "Follow a path through the tree and return the oddball that the tree leads us to."
    if is_leaf(tree):
        return tree
    else:
        branch = weigh(tree.L, tree.R, oddball)
        return follow(getattr(tree, branch), oddball)
    
def valid(tree, puzzle) -> bool:
    "Does the strategy tree solve the puzzle correctly for all possible oddballs?"
    return (tree is not None and
            depth(tree) <= puzzle.weighings and 
            all(follow(tree, oddball) == oddball 
                for oddball in puzzle.oddballs))

def depth(tree) -> int:
    "Maximum depth (number of weighings) in tree."
    return (0 if is_leaf(tree) or tree is None
            else 1 + max(depth(tree.gt), depth(tree.eq), depth(tree.lt)))

def first(iterable): return next(iter(iterable))

def is_leaf(tree): return isinstance(tree, Leaf)

Let's try out some of these functions:

In [3]:
# A puzzle with 8 balls and 3 weighings allowed
p8 = Puzzle(8, 3) 
p8.weighings

3

In [4]:
p8.balls 

[1, 2, 3, 4, 5, 6, 7, 8]

In [5]:
p8.oddballs

{-8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8}

In [6]:
# If we weigh balls 1, 2 against 3, 4, and 4 is lighter, the result should be 'gt'
weigh([1, 2], [3, 4], -4)

'gt'

In [7]:
weight(1, oddball=-4)

100

In [8]:
weight(4, oddball=-4)

99

# Finding a Valid Tree

Now we've got all the pieces we need to find a valid tree to solve a puzzle. The key concept is that a **weighing** gives us information by making a **partition** of the possible **oddballs** into branches for each of the three possible weighing results: `gt`, `eq`, or `lt`. If each of the partitions is small enough, subsequent weighings can handle each of them. Here's what I mean by a partition:

In [9]:
def partition(L, R, oddballs) -> dict:
    "Build a dict that partitions the possible outcomes (oddballs) resulting from weighing L versus R."
    part = dict(gt=set(), eq=set(), lt=set())
    for odd in oddballs:
        part[weigh(L, R, odd)].add(odd)
    return part

For example, with 12 balls, if we weigh balls {1, 2, 3} on the left versus {10, 11, 12} on the right, then there are 6 ways  the left side can be greater than the right: either 10, 11 or 12 is lighter or  1, 2, or 3 is heavier. Similarly there are 6 ways of getting a `lt` weighing result. The remaining 12 possibilities&mdash;balls 4 through 9 being either heavier or lighter&mdash;show up in the `eq` branch of the partition:

In [10]:
p12 = Puzzle(12, 3)

partition([1, 2, 3], [10, 11, 12], p12.oddballs)

{'gt': {-12, -11, -10, 1, 2, 3},
 'eq': {-9, -8, -7, -6, -5, -4, 4, 5, 6, 7, 8, 9},
 'lt': {-3, -2, -1, 10, 11, 12}}

If this was the first weighing in our strategy tree, could we go on to solve the puzzle in 3 weighings? **No!** With two remaining weighings we can handle at most  3 &times; 3 = 9 possibilities; here we have 12 possibilities in the `eq` branch, which is too many. We call this a **bad partition**.

The following is a **good partition** because each of the branches has 8 possibilities, and 8 is less than 9. 

In [11]:
partition([1, 2, 3, 4], [9, 10, 11, 12], p12.oddballs)

{'gt': {-12, -11, -10, -9, 1, 2, 3, 4},
 'eq': {-8, -7, -6, -5, 5, 6, 7, 8},
 'lt': {-4, -3, -2, -1, 9, 10, 11, 12}}

(Note: being a good partition does not guarantee that the puzzle is solvable from there; but being a bad partition guarantees that it is not solvable. Also note that this is a good partition with 2 remaining weighings, but would be a bad partition with only one remaining weighing.)

So now we have a viable approach to implementing `find_tree`:

   - We call `find_tree(puzzle, oddballs, weighings)`. At the top level, the oddballs and number of weighings come from the puzzle. At recursive levels, we will reduce the number of oddball possibilities according to the partition, and the number of remaining weighings by 1 each time.
   - We handle trivial cases when there are zero or one oddballs remaining, or no weighings remaining.
   - Otherwise, we generate random partitions of the balls, and examine the good ones.
   - Given a good partition, all we have to do is find good trees (i.e. not `None`) for each of the three branches. If we can find that, return the tree as a solution.
   - If we can't find good trees for any partition, return `None` to indicate failure.
   
The rest of the work is done by `good_partitions`:

   - We select two lists of balls to be weighed, `L` and `R`, both of length `B`.
   - We systematically consider all reasonable values of `B`. (All values from 1 up to 1/3 the number of balls, rounded up.)
   - We then see what partition `L` and `R` gives us.
   - If it is good, the function yields the `(L, R, partition)` tuple.
   - In any case, we shuffle the balls and repeat, trying to generate more good partitions. I chose to repeat 100 times, to have a very good chance of randomly selecting a good partition for solvable puzzles, and of terminating in just a few seconds for unsolvable puzzles. 
   - However, I note that on the first weighing, there's no sense making multiple tries with randomly shuffled  balls&mdash;the only choice that matters is how many balls, `B`, to put on each side, because the balls are undifferentiated at this point. So I only repeat for one try in that case.
   - If I wanted to solve puzzles with hundreds or  thousands of balls, not just dozens, I would come back and improve `good_partitions`.

In [12]:
def find_tree(puzzle, oddballs, weighings) -> Tree or None:
    "Find a strategy tree that covers all the oddballs in the given number of weighings."
    if len(oddballs) == 1:
        return first(oddballs) # One possibility left; we're done; leaf node
    elif len(oddballs) == 0:
        return 0               # No oddball (a possibility for some puzzles)
    elif weighings == 0:
        return None            # No solution; failure
    else:
        for L, R, part in good_partitions(puzzle, oddballs, weighings - 1):
            subtrees = {r: find_tree(puzzle, part[r], weighings - 1) for r in part}
            if None not in subtrees.values(): 
                return Node(L, R, **subtrees)
    
def good_partitions(puzzle, oddballs, weighings, tries=100):
    "Yield random (L, R, partition) tuples with no partition branch longer than 3**weighings."
    balls = sorted(puzzle.balls)
    for repeat in range(1 if oddballs == puzzle.oddballs else tries): 
        for B in range(1, (len(balls) + 2) // 3 + 1):
            L, R = balls[:B], balls[-B:]
            part = partition(L, R, oddballs)
            longest_part = max(part.values(), key=len)
            if len(longest_part) <= 3 ** weighings:
                yield L, R, part
        random.shuffle(balls)

Here we see that  `good_partitions` does its job:

In [13]:
next(good_partitions(p12, p12.oddballs, 2))

([1, 2, 3, 4],
 [9, 10, 11, 12],
 {'gt': {-12, -11, -10, -9, 1, 2, 3, 4},
  'eq': {-8, -7, -6, -5, 5, 6, 7, 8},
  'lt': {-4, -3, -2, -1, 9, 10, 11, 12}})

# Solving Some Puzzles

Now we're ready to solve puzzles!

In [14]:
solve(p12)

Node(L=[1, 2, 3, 4], R=[9, 10, 11, 12], gt=Node(L=[3, 12, 7], R=[9, 2, 10], gt=Node(L=[4, 10, 3], R=[8, 1, 2], gt=3, eq=-9, lt=-10), eq=Node(L=[5, 12, 6, 9], R=[4, 7, 11, 3], gt=-11, eq=1, lt=4), lt=Node(L=[1], R=[12], gt=-12, eq=2, lt=0)), eq=Node(L=[7, 4, 11, 6], R=[1, 10, 9, 8], gt=Node(L=[7, 12], R=[6, 1], gt=7, eq=-8, lt=6), eq=Node(L=[12], R=[5], gt=-5, eq=0, lt=5), lt=Node(L=[4, 5], R=[8, 7], gt=-7, eq=-6, lt=8)), lt=Node(L=[7, 6, 9, 3], R=[10, 2, 4, 5], gt=Node(L=[12, 10], R=[2, 9], gt=-2, eq=-4, lt=9), eq=Node(L=[11, 6, 8, 1], R=[9, 4, 7, 5], gt=11, eq=12, lt=-1), lt=Node(L=[1, 10], R=[2, 11], gt=10, eq=-3, lt=0)))

OK, that's hard to read&mdash;my bad. Let's look at a much easier puzzle: 3 balls, 1 weighing allowed, and the oddball can only be lighter:

In [15]:
tree = solve(Puzzle(3, 1, odd={-1}))
tree

Node(L=[1], R=[3], gt=-3, eq=-2, lt=-1)

This tree says you weigh the first ball against the third (leaving the second unweighed), and the three possible weighing results tell you which of the three balls  is lighter. 

# Prettier Output

Let's make the output easier to read. I'll use `[1·2·3·4]⚖[9·10·11·12] ➔` to mean "*Weigh balls 1,2,3,4 versus 9,10,11,12 to get a result...*". I'll indent each interior node in the tree, and I'll use `>:` to mean *the result when the left hand side is greater than the right in weight is...*  

In [16]:
def do(puzzle):
    "Print the solution to the puzzle as indented text."
    tree = solve(puzzle)
    print(indented(tree))
    
def indented(tree, i=0, prefix='') -> str:
    "Pretty, indented string representing a strategy tree."
    if tree == 0 or tree == None:
        return f'{prefix}{tree}'
    elif is_leaf(tree):
        return f'{prefix}{tree:+d}'
    else:
        subtrees = (indented(tree.gt, i+1, '>:'), 
                    indented(tree.eq, i+1, '=:'), 
                    indented(tree.lt, i+1, '<:'))
        indent   = ('' if i == 0 else ('\n' + ' ' * 5 * i))
        return f'{indent}{prefix}{side(tree.L)}⚖{side(tree.R)} ➔ {" ".join(subtrees)})'
    
def side(balls) -> str: return str(sorted(balls)).replace(', ', '·')

In [17]:
# 3 balls, 1 weighing, only lighter balls possible
do(Puzzle(3, 1, {-1}))

[1]⚖[3] ➔ >:-3 =:-2 <:-1)


In [18]:
# 3 balls, 2 weighings, lighter or heavier balls possible
do(Puzzle(3, 2))

[1]⚖[3] ➔ 
     >:[2]⚖[3] ➔ >:-3 =:+1 <:0) 
     =:[2]⚖[1] ➔ >:+2 =:0 <:-2) 
     <:[1]⚖[2] ➔ >:0 =:+3 <:-1))


In [19]:
# The original puzzle with 12 balls, 3 weighings
do(p12)

[1·2·3·4]⚖[9·10·11·12] ➔ 
     >:[1·3·10·12]⚖[4·6·7·9] ➔ 
          >:[1·9]⚖[10·11] ➔ >:+1 =:+3 <:-9) 
          =:[2·9]⚖[10·12] ➔ >:+2 =:-11 <:0) 
          <:[4·8·12]⚖[1·2·6] ➔ >:+4 =:-10 <:-12)) 
     =:[3·6·8]⚖[1·4·7] ➔ 
          >:[5·6]⚖[8·9] ➔ >:+6 =:-7 <:+8) 
          =:[1·2·11]⚖[5·8·10] ➔ >:-5 =:0 <:+5) 
          <:[6·7·9]⚖[2·5·12] ➔ >:+7 =:-8 <:-6)) 
     <:[1·3·9·11]⚖[2·7·8·12] ➔ 
          >:[5·12]⚖[2·11] ➔ >:-2 =:+9 <:+11) 
          =:[1·2·3]⚖[10·11·12] ➔ >:0 =:-4 <:+10) 
          <:[1·2·7]⚖[3·8·9] ➔ >:-3 =:+12 <:-1)))


In [20]:
# A different solution to the same puzzle
do(p12)

[1·2·3·4]⚖[9·10·11·12] ➔ 
     >:[3·4·6·12]⚖[2·5·8·9] ➔ 
          >:[1·8]⚖[3·9] ➔ >:-9 =:+4 <:+3) 
          =:[2·10]⚖[7·11] ➔ >:-11 =:+1 <:-10) 
          <:[1]⚖[12] ➔ >:-12 =:+2 <:0)) 
     =:[2·5·6]⚖[3·8·11] ➔ 
          >:[4·5·7·8]⚖[1·2·3·10] ➔ >:+5 =:+6 <:-8) 
          =:[1·5·8·12]⚖[3·4·7·11] ➔ >:-7 =:0 <:+7) 
          <:[3·7·11]⚖[5·8·10] ➔ >:-5 =:-6 <:+8)) 
     <:[3·4·5·11]⚖[1·2·7·10] ➔ 
          >:[5·7·8]⚖[2·6·11] ➔ >:-2 =:-1 <:+11) 
          =:[1]⚖[12] ➔ >:0 =:+9 <:+12) 
          <:[3·5·11·12]⚖[4·6·7·9] ➔ >:-4 =:+10 <:-3)))


I note that the traditional answer to the puzzle weighs 4 balls on each side of the first weighing, three balls on the second weighing, and 2 balls on the third weighing. But my program is not so orderly. It always weighs 4 balls on the first weighing, but from there it can vary; it has no inclination to minimize the number of balls on each side of the scale, or to make one branch of the tree be similar to another branch.

# Other Puzzles: 3 Weighings



We can do **12 balls in 3 weighings** even when we add in the possibility that all the balls weigh the same (we use `0` to denote this situation):

In [21]:
do(Puzzle(12, 3, odd={-1, 0, +1}))

[1·2·3·4]⚖[9·10·11·12] ➔ 
     >:[3·4·10]⚖[1·6·12] ➔ 
          >:[3·8]⚖[4·5] ➔ >:+3 =:-12 <:+4) 
          =:[3·4·5]⚖[2·9·10] ➔ >:-9 =:-11 <:+2) 
          <:[1]⚖[12] ➔ >:+1 =:-10 <:0)) 
     =:[5·7]⚖[8·11] ➔ 
          >:[3·6·7·8]⚖[1·10·11·12] ➔ >:+7 =:+5 <:-8) 
          =:[6·8·10]⚖[1·9·11] ➔ >:+6 =:0 <:-6) 
          <:[1·6·12]⚖[5·8·11] ➔ >:-5 =:-7 <:+8)) 
     <:[4·5·9·12]⚖[7·8·10·11] ➔ 
          >:[1]⚖[12] ➔ >:0 =:+9 <:+12) 
          =:[3]⚖[1] ➔ >:-1 =:-2 <:-3) 
          <:[1·2·5·12]⚖[3·4·6·11] ➔ >:-4 =:+10 <:+11)))


Can we solve the **13-balls in 3 weighings** puzzle, which has 26 possibilities? After all, 26 is less than 3<sup>3</sup>.

In [22]:
do(Puzzle(13, 3))

None


**No**, and the reason is that there is no first weighing that partitions the 26 possibilities into 9/9/8; the best we can do is partition into 8/10/8 or 10/6/10, and 10 possibilities can't be distinguished in the remaining two weighings:

In [23]:
partition([1, 2, 3, 4], [10, 11, 12, 13], Puzzle(13).oddballs)

{'gt': {-13, -12, -11, -10, 1, 2, 3, 4},
 'eq': {-9, -8, -7, -6, -5, 5, 6, 7, 8, 9},
 'lt': {-4, -3, -2, -1, 10, 11, 12, 13}}

In [24]:
partition([1, 2, 3, 4, 5], [9, 10, 11, 12, 13], Puzzle(13).oddballs)

{'gt': {-13, -12, -11, -10, -9, 1, 2, 3, 4, 5},
 'eq': {-8, -7, -6, 6, 7, 8},
 'lt': {-5, -4, -3, -2, -1, 9, 10, 11, 12, 13}}

We can do **27 balls in 3 weighings** if we know that the oddball can only be lighter, not heavier:

In [25]:
do(Puzzle(27, 3, {-1}))

[1·2·3·4·5·6·7·8·9]⚖[19·20·21·22·23·24·25·26·27] ➔ 
     >:[10·14·18·21·24·25]⚖[8·13·15·19·20·27] ➔ 
          >:[25·27]⚖[18·19] ➔ >:-19 =:-20 <:-27) 
          =:[12·13·26]⚖[17·23·24] ➔ >:-23 =:-22 <:-26) 
          <:[2·5·11·24]⚖[14·20·22·25] ➔ >:-25 =:-21 <:-24)) 
     =:[3·8·13·14·18·19·20·23·25]⚖[1·7·10·11·16·21·22·24·27] ➔ 
          >:[2·13·16·20·22·25]⚖[4·10·17·18·23·27] ➔ >:-10 =:-11 <:-16) 
          =:[9·17·24·26]⚖[1·3·5·12] ➔ >:-12 =:-15 <:-17) 
          <:[1·18·20·21·23]⚖[6·13·15·22·25] ➔ >:-13 =:-14 <:-18)) 
     <:[2·3·8·10·11·16·20·25·27]⚖[1·4·5·15·17·18·21·22·23] ➔ 
          >:[3·5·9·10·14·19·22·27]⚖[2·4·6·7·13·15·17·25] ➔ >:-4 =:-1 <:-5) 
          =:[1·7·10·11·27]⚖[9·13·21·22·26] ➔ >:-9 =:-6 <:-7) 
          <:[1·4·6·8·9·13·14·15·26]⚖[3·5·16·18·19·20·21·23·27] ➔ >:-3 =:-2 <:-8)))


And we can do **26 balls** under the condition that either one ball is lighter or all the balls weigh the same:

In [26]:
do(Puzzle(26, 3, {-1, 0}))

[1·2·3·4·5·6·7·8·9]⚖[18·19·20·21·22·23·24·25·26] ➔ 
     >:[1·2·6·9·13·17·22·23·26]⚖[4·5·11·14·15·16·18·20·24] ➔ 
          >:[1·2·9·15·19·22·24·26]⚖[6·8·10·11·13·14·16·18] ➔ >:-18 =:-20 <:-24) 
          =:[1·4·6·8·11·14·15·23·25]⚖[3·7·10·13·16·17·21·24·26] ➔ >:-21 =:-19 <:-25) 
          <:[4·10·13·18·19·26]⚖[2·7·8·15·17·23] ➔ >:-23 =:-22 <:-26)) 
     =:[5·6·12·14·16·18·22·24]⚖[1·13·15·17·19·23·25·26] ➔ 
          >:[3·6·8·12·14·17·22·23·25]⚖[2·4·5·9·13·18·19·21·24] ➔ >:-13 =:-15 <:-17) 
          =:[10·16·23]⚖[11·19·24] ➔ >:-11 =:0 <:-10) 
          <:[6·11·12·19·21·22·25]⚖[1·7·13·14·17·23·26] ➔ >:-14 =:-16 <:-12)) 
     <:[4·5·8·10·13·14·18]⚖[2·7·9·17·20·24·25] ➔ 
          >:[8·9·13·14·17·18·20·23·25]⚖[1·3·5·6·7·10·16·22·26] ➔ >:-7 =:-2 <:-9) 
          =:[1·9·11·15·20]⚖[6·7·8·17·25] ➔ >:-6 =:-3 <:-1) 
          <:[8·12·15·18·23]⚖[1·5·11·16·26] ➔ >:-5 =:-4 <:-8)))


Here's a puzzle with **25 balls** with an unusual set of possibilities: either one of the odd-numbered balls is heavier, or one of the even-numbered balls is lighter, or no ball is odd.

In [27]:
N = 25
p = Puzzle(N, 3, oddballs={(+b if b % 2 else -b) for b in range(N + 1)})
print(p.oddballs)

{0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, -24, -22, -20, -18, -16, -14, -12, -10, -8, -6, -4, -2}


In [28]:
do(p)

[1·2·3·4·5·6·7·8·9]⚖[17·18·19·20·21·22·23·24·25] ➔ 
     >:[2·3·4·6·8·11·12·22·23]⚖[1·5·10·15·16·18·21·24·25] ➔ 
          >:[3·14·16·18·21]⚖[6·11·17·20·23] ➔ >:+3 =:-24 <:-18) 
          =:[1·4·7·8·10·12·20·23·24]⚖[2·3·5·6·16·17·18·19·22] ➔ >:+7 =:+9 <:-20) 
          <:[2·4·5·6·9·10·13·16·18]⚖[1·3·7·12·14·15·19·20·25] ➔ >:+5 =:-22 <:+1)) 
     =:[6·7·10·11·13·14·18]⚖[1·4·8·9·16·20·23] ➔ 
          >:[1·4·6·13·16·18·19·20·25]⚖[3·5·7·8·9·10·12·14·17] ➔ >:+13 =:+11 <:-16) 
          =:[8·14·25]⚖[12·15·18] ➔ >:-12 =:0 <:+15) 
          <:[2·5]⚖[10·24] ➔ >:-10 =:-14 <:0)) 
     <:[1·6·11·14·24·25]⚖[4·5·8·13·19·21] ➔ 
          >:[4·10·12·17·22·25]⚖[2·13·14·16·21·24] ➔ >:+25 =:-8 <:-4) 
          =:[9·12·14·18·22·25]⚖[1·2·5·20·23·24] ➔ >:-2 =:+17 <:+23) 
          <:[4·6·15·18·19]⚖[2·8·9·11·12] ➔ >:+19 =:+21 <:-6)))


Another variation: **27 balls, 3 weighings**, the oddball can only be heavier, but one ball (number 27) is poisonous and can't be touched or placed on the balance scale. It might, however be the heavier ball, and you still need to report it as such if it is. We describe this situation by defining a puzzle with 26 (weighable) balls, but with the oddballs covering `{+1, +2, ..., +27}`. Note that when all three weighings are equal, the strategy tree reports `=:+27`.

In [29]:
do(Puzzle(26, 3, oddballs=range(1, 28)))

[1·2·3·4·5·6·7·8·9]⚖[18·19·20·21·22·23·24·25·26] ➔ 
     >:[3·5·8·11·18·19]⚖[4·6·7·16·20·26] ➔ 
          >:[4·8·15]⚖[5·20·24] ➔ >:+8 =:+3 <:+5) 
          =:[2·4·6·11·19·24]⚖[1·10·13·17·20·22] ➔ >:+2 =:+9 <:+1) 
          <:[1·4·13·20·22·24]⚖[6·8·9·16·18·19] ➔ >:+4 =:+7 <:+6)) 
     =:[1·8·10·11·17·22·24]⚖[3·6·7·13·14·15·26] ➔ 
          >:[10]⚖[17] ➔ >:+10 =:+11 <:+17) 
          =:[2·5·11·14·16·18·19·26]⚖[3·4·12·13·17·20·22·24] ➔ >:+16 =:+27 <:+12) 
          <:[2·3·8·9·15·18·20·25]⚖[1·7·10·14·16·17·21·22] ➔ >:+15 =:+13 <:+14)) 
     <:[1·6·9·11·14·17·19·21·22]⚖[2·3·5·10·12·16·18·23·26] ➔ 
          >:[19]⚖[22] ➔ >:+19 =:+21 <:+22) 
          =:[1·10·11·13·19·20·26]⚖[3·9·14·15·21·22·25] ➔ >:+20 =:+24 <:+25) 
          <:[2·4·13·15·26]⚖[3·16·18·19·20] ➔ >:+26 =:+23 <:+18)))


# 4 Weighings

We can tackle larger puzzles. With 4 weighings, we can theoretically handle up to 3<sup>4</sup> = 81 possibilities. Can we solve for **40 balls**?

In [30]:
p40 = Puzzle(40, 4)

do(p40)

None


Unfortunately, **no**. We always end up with at least one branch with 28 possibilities, but we need 27 or less:

In [31]:
def partition_sizes(puzzle, B):
    "How big is each branch of a partition with B balls on both sides?"
    part = partition(puzzle.balls[:B], puzzle.balls[-B:], puzzle.oddballs)
    return [len(part[branch]) for branch in part]

In [32]:
partition_sizes(p40, 13)

[26, 28, 26]

In [33]:
partition_sizes(p40, 14)

[28, 24, 28]

How about **39 balls**, with the possibility that no ball is odd. That's 39 &times; 2 + 1 = 79 possibilities, and it turns out we can find a good partition, and a solution:

In [34]:
p39 = Puzzle(39, 4, {-1, 0, +1})

In [35]:
partition_sizes(p39, 13)

[26, 27, 26]

In [36]:
do(p39)

[1·2·3·4·5·6·7·8·9·10·11·12·13]⚖[27·28·29·30·31·32·33·34·35·36·37·38·39] ➔ 
     >:[3·9·10·14·15·21·24·26·28·30·31·36·38]⚖[2·6·8·11·16·17·20·27·29·32·34·37·39] ➔ 
          >:[4·6·13·18·21·23·24·25·27·28·30·36]⚖[1·2·3·10·11·15·19·26·29·32·35·39] ➔ 
               >:[7·11·20·22·32·35·37]⚖[4·6·19·21·31·33·39] ➔ >:-39 =:-29 <:-32) 
               =:[1·4·5·6·13·19·21·26·27·29·31·36]⚖[2·9·10·12·15·20·24·28·32·34·35·38] ➔ >:-34 =:-37 <:+9) 
               <:[1·3·18·19·21·23·27·36·37]⚖[2·5·9·12·13·20·25·26·31] ➔ >:+3 =:+10 <:-27)) 
          =:[6·7·8·11·14·21·26·27·32·35]⚖[12·13·19·22·23·24·31·33·34·39] ➔ 
               >:[10·11·21·22·24·25·30·33·36·37·39]⚖[4·6·8·9·16·20·23·26·29·31·35] ➔ >:0 =:+7 <:-33) 
               =:[1·9·16·17·20·22·30·37·39]⚖[4·8·11·18·19·24·28·29·36] ➔ >:+1 =:+5 <:+4) 
               <:[9·13·25·27·33·35·36]⚖[1·5·6·11·22·30·31] ➔ >:+13 =:+12 <:-35)) 
          <:[3·5·6·11·13·19·25·26·28·36]⚖[2·10·15·17·20·23·29·30·34·37] ➔ 
               >:[5·11·15·16·27·29·31·32·38]

How about **80 balls** under the condition that no ball can be heavier (thus, 81 possibilities):

In [37]:
do(Puzzle(80, 4, {-1, 0}))

[1·2·3·4·5·6·7·8·9·10·11·12·13·14·15·16·17·18·19·20·21·22·23·24·25·26·27]⚖[54·55·56·57·58·59·60·61·62·63·64·65·66·67·68·69·70·71·72·73·74·75·76·77·78·79·80] ➔ 
     >:[8·10·19·25·26·27·28·35·38·41·42·45·47·48·56·57·58·60·62·68·69·73·75]⚖[2·3·7·9·14·18·22·29·31·36·39·40·50·53·54·61·63·64·70·72·77·78·80] ➔ 
          >:[2·4·13·14·16·22·23·24·25·26·30·38·40·47·48·49·50·53·56·58·60·63·68·69·70·75·78]⚖[1·3·5·7·8·12·21·27·28·29·34·36·37·39·41·42·43·45·51·54·55·57·62·66·72·76·80] ➔ 
               >:[10·11·21·23·26·28·36·77·80]⚖[27·34·37·40·53·57·63·65·72] ➔ >:-72 =:-54 <:-80) 
               =:[2·6·7·24·27·31·34·41·60·62·71·77]⚖[1·12·13·18·23·46·51·53·63·64·70·80] ➔ >:-64 =:-61 <:-77) 
               <:[7·11·20·21·24·28·29·30·33·39·43·52·59·61·62·65·70·75·79·80]⚖[2·4·14·15·16·18·36·38·42·44·46·49·50·56·64·67·69·71·72·78] ➔ >:-78 =:-63 <:-70)) 
          =:[1·3·15·18·19·23·25·27·28·29·30·33·38·42·44·47·49·60·62·63·64·66·68·70·74·76]⚖[2·4·7·11·12·13·14·16·22·35·39·43·45·46·48·50·51·52·56·58·59

Looking good (except that the output is longer than the line length, and so again is hard to read).

# 5 Weighings

With 5 weighings, 3<sup>5</sup> = 243, so I might be able to handle up to 121 lighter-or-heavier balls (242 possibilities), if I can make partitions of size 81 or less. Let's try:

In [38]:
p5 = Puzzle(121, 5)
do(p5)

None


In [39]:
partition_sizes(p5, 40)

[80, 82, 80]

In [40]:
partition_sizes(p5, 41)

[82, 78, 82]

Nope, that didn't work. Let's back off to 2 &times; 120 + 1 = 241 possibilities:

In [41]:
do(Puzzle(120, 5, {-1, 0, +1}))

[1·2·3·4·5·6·7·8·9·10·11·12·13·14·15·16·17·18·19·20·21·22·23·24·25·26·27·28·29·30·31·32·33·34·35·36·37·38·39·40]⚖[81·82·83·84·85·86·87·88·89·90·91·92·93·94·95·96·97·98·99·100·101·102·103·104·105·106·107·108·109·110·111·112·113·114·115·116·117·118·119·120] ➔ 
     >:[1·2·6·7·9·13·16·18·19·20·22·27·28·33·35·44·48·50·61·64·65·69·71·87·89·91·92·96·97·99·100·101·102·106·119·120]⚖[3·5·8·10·11·12·21·25·29·31·34·36·39·40·43·46·52·53·55·57·62·67·68·74·80·82·93·98·104·107·110·111·113·115·116·118] ➔ 
          >:[1·4·12·15·19·20·25·27·30·36·37·42·47·48·49·53·55·60·64·65·66·69·76·78·81·90·94·96·101·102·108·109·110·112]⚖[5·9·13·16·17·18·21·22·26·33·35·39·40·44·50·56·61·71·72·73·75·80·85·86·87·95·97·98·103·104·111·113·115·117] ➔ 
               >:[1·6·9·14·23·33·37·38·43·45·47·49·59·61·67·69·73·79·80·81·82·84·88·93·98·99·100·101·102·107·110·114]⚖[3·11·16·17·18·19·25·26·27·29·31·32·34·35·42·48·52·57·58·60·66·68·71·72·76·83·90·91·104·106·113·119] ➔ 
                    >:[22·25·28·34·36·42·43·75·78·88

That works.

Or, I could solve a puzzle with 243 possibilites (the maximum possible with 5 weighings): 242 lighter-only balls, plus the possibility that no ball is odd:

In [42]:
do(Puzzle(242, 5, {-1, 0}))

[1·2·3·4·5·6·7·8·9·10·11·12·13·14·15·16·17·18·19·20·21·22·23·24·25·26·27·28·29·30·31·32·33·34·35·36·37·38·39·40·41·42·43·44·45·46·47·48·49·50·51·52·53·54·55·56·57·58·59·60·61·62·63·64·65·66·67·68·69·70·71·72·73·74·75·76·77·78·79·80·81]⚖[162·163·164·165·166·167·168·169·170·171·172·173·174·175·176·177·178·179·180·181·182·183·184·185·186·187·188·189·190·191·192·193·194·195·196·197·198·199·200·201·202·203·204·205·206·207·208·209·210·211·212·213·214·215·216·217·218·219·220·221·222·223·224·225·226·227·228·229·230·231·232·233·234·235·236·237·238·239·240·241·242] ➔ 
     >:[4·5·6·11·28·29·31·33·35·36·40·43·44·46·48·49·52·56·57·65·66·72·73·80·81·84·87·89·91·96·101·105·106·110·116·118·124·126·127·128·130·132·133·141·142·143·147·149·150·151·154·157·160·164·171·172·173·175·186·194·195·198·200·201·202·205·206·207·208·211·213·214·218·220·226·228·229·231·234·240]⚖[1·7·8·10·12·13·15·16·20·21·22·23·24·26·27·30·32·39·42·45·47·51·54·60·61·62·64·67·70·71·74·78·85·86·88·90·93·94·97·104·108·111·113·117·120·

# What's Next?

- What other puzzles can you solve?
- Can you make a table summarizing the space of solved and unsolved puzzles? 
- Can you prove the unsolved puzzles are unsolvable? 
- Can you prove that it is never necessary to put more than 1/3 the number of balls (rounded up) on either side of the scale?
- What happens when it is a possibility that *two* balls are odd?  Or more?
- What about puzzles where your task is to identify *which* ball is odd, but not *how* it is odd? That is, if you get the possibilities down to `{-3, +3}` then you're done; you know ball 3 is the odd one, and you don't care if it is heavy or light.
- Currently, when `solve` returns `None`, it means "no solution was found, but there's no proof that a solution doesn't exist." Can you modify `solve` to prove there is no solution for all (or at least some) puzzles?
- Can you find trees that minimize the *mean* number of weighings, rather than minimizing the *maximum* number of weighings?
- What if you had two or more balance scales that you could use in parallel, and the goal was to minimize the number of weighing time periods, not the total number of weighings?
- What else can you discover?